In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import pyswarms as ps



In [6]:
datasetPath = "Training_Dataset.csv"
dataset_data = pd.read_csv(datasetPath, sep=',', header=0, na_filter=True)
y = dataset_data.iloc[:,-1]
y = y.values
X = dataset_data.drop(labels='Result', axis=1)
X = X.values

In [7]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
   
clf=RandomForestClassifier(n_estimators=50,max_depth =5)

from sklearn import metrics

clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, predictions)
print(accuracy)

0.9315646668676515


In [8]:
clf = linear_model.LogisticRegression()


def f_per_particle(m, alpha):
    total_features = X.shape[1]
    if np.count_nonzero(m) == 0:
        X_subset = X
    else:
        X_subset = X[:,m==1]

    clf.fit(X_subset, y)
    P = (clf.predict(X_subset) == y).mean()
    j = (alpha * (1.0 - P)
        + (1.0 - alpha) * (1 - (X_subset.shape[1] / total_features)))
    return j

In [9]:
def f(x, alpha=0.9):
    n_particles = x.shape[0]
    j = [f_per_particle(x[i], alpha) for i in range(n_particles)]
    return np.array(j)

In [10]:
#!pip install pyswarms

In [15]:
from datetime import datetime as dt
import time

start = dt.now()
print("Started at: ", str(start))
particleScore = list()
particleSize = list()

options = {'c1': 2, 'c2': 2, 'w':0.3, 'k': 20, 'p':2}


dimensions = X.shape[1]
optimizer = ps.discrete.BinaryPSO(n_particles= 30, dimensions=dimensions, options=options)


cost, pos = optimizer.optimize(f, iters=25, verbose=2)

end = dt.now()
print("Finished at: ", str(end))
total = end-start
print("Total time spent: ", total)


2021-03-27 12:57:36,574 - pyswarms.discrete.binary - INFO - Optimize for 25 iters with {'c1': 2, 'c2': 2, 'w': 0.3, 'k': 20, 'p': 2}
pyswarms.discrete.binary:   0%|                                                                                   |0/25

Started at:  2021-03-27 12:57:36.564419


pyswarms.discrete.binary: 100%|████████████████████████████████████████████████████████████████|25/25, best_cost=0.0918
2021-03-27 12:59:44,176 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.09180009045680684, best pos: [1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0]


Finished at:  2021-03-27 12:59:44.178965
Total time spent:  0:02:07.614546


In [14]:

X_selected_features = X[:,pos==1]
print(len(X_selected_features[0]))

X_train, X_test, y_train, y_test = train_test_split(X_selected_features, y, test_size=0.3)

clf2 = RandomForestClassifier(n_estimators =50,max_depth =15)
     

from sklearn import metrics

clf2.fit(X_train, y_train)
predictions = clf2.predict(X_test)
accuracy = metrics.accuracy_score(y_test, predictions)
print(accuracy)

20
0.9541754597527886


In [17]:
print(X_selected_features)
np.savetxt('2darray.csv', X_selected_features, delimiter=',', fmt='%d')

[[ 1  1 -1 ... -1  1 -1]
 [ 1  1 -1 ... -1  1  1]
 [ 0  1 -1 ... -1  1 -1]
 ...
 [-1  1 -1 ... -1  1  1]
 [-1  1 -1 ... -1  1  1]
 [-1  1 -1 ... -1 -1 -1]]
